In [136]:
# Puzzle 1
cards = {"2" : 2,
         "3" : 3, 
         "4" : 4,
         "5" : 5,
         "6" : 6,
         "7" : 7,
         "8" : 8,
         "9" : 9,
         "T" : 10,
         "J" : 11,
         "Q" : 12,
         "K" : 13,
         "A" : 14}

cards_J = {"2" : 2,
         "3" : 3, 
         "4" : 4,
         "5" : 5,
         "6" : 6,
         "7" : 7,
         "8" : 8,
         "9" : 9,
         "T" : 10,
         "J" : 1,   # Joker extra rule
         "Q" : 12,
         "K" : 13,
         "A" : 14}

type_strength = {"5K" : 7,
                 "4K" : 6,
                 "FH" : 5,
                 "3K" : 4,
                 "2P" : 3,
                 "1P" : 2,
                 "HC" : 1}


def get_data(filename):
    d_hands = {}
    with open(filename, "r") as file:
        for line in file:
            line = line.strip()
            hand, bid = line.split(" ")
            bid = int(bid)
            d = {"bid" : bid}
            d_hands[hand] = d
    return d_hands

def sort_hand(hand):
    global cards
    l_c = []
    for card in hand:
        l_c.append(cards[card])
    l_c.sort()
    sorted_hand = ""
    for char in l_c: 
        # looks for the key which fits to the value in char
        sorted_hand += list(cards.keys())[list(cards.values()).index(char)]
    return sorted_hand
    
def add_sort_hands(d_hands):
    global cards
    for hand in d_hands:
        d_hands[hand]["sor"] = sort_hand(hand)
    return d_hands
        
def find_type(card):
    a, b, c, d, e = card
    if a == b and b == c and c == d and d == e:
        return "5K"
    elif (a == b and b == c and c == d) or (b == c and c == d and d == e):
        return "4K"
    elif ((a == b and b == c) and (d == e)) or ((a == b) and (c == d and d == e)):
        return "FH"
    elif (a == b and b == c) or (b == c and c == d) or (c == d and d == e):
        return "3K"
    elif (a == b and c == d) or (a == b and d == e) or (b == c and d == e):
        return "2P"
    elif (a == b) or (b == c) or (c == d) or (d == e):
        return "1P"
    else:
        return "HC"
    
def add_type(d_hands):
    for hand in d_hands:    
        d_hands[hand]["type"] = find_type(d_hands[hand]["sor"])     
    return d_hands

def add_strength(d_hands):
    global type_strength
    for hand in d_hands:
        d_hands[hand]["strength"] = type_strength[d_hands[hand]["type"]]
    return d_hands

def max_of_two_hands(hand1, hand2, strength1, strength2, joker):
    global cards
    global cards_J
    if joker:
        crds = cards_J
    else:
        crds = cards
    if strength1 > strength2:
        return hand1
    elif strength1 < strength2:
        return hand2
    else:
        for i in range(5):
            if crds[hand1[i]] > crds[hand2[i]]:
                return hand1
            elif crds[hand1[i]] < crds[hand2[i]]:
                return hand2
    return hand1

def add_rank(d_hands, joker):
    l_hands = []
    for hand in d_hands:
        l_hands.append(hand)
    l_ranks = []
    while (len(l_hands) > 0):
        max_hand = l_hands[0]
        for hand in l_hands:
            max_strength = d_hands[max_hand]["strength"]
            hand_strength = d_hands[hand]["strength"]
            max_hand = max_of_two_hands(max_hand, hand, max_strength, hand_strength, joker)
        l_ranks.append(max_hand)
        l_hands.remove(max_hand)
    #print(l_ranks)
    for hand in l_ranks:
        rank = len(l_ranks) - l_ranks.index(hand)
        d_hands[hand]["rank"] = rank
    #print(l_ranks)
    return d_hands

def optimize_hand(hand, d_hands):
    global cards
    global type_strength
    opt_hand = hand 
    opt_strength = d_hands[opt_hand]["strength"]
    for card in cards:
        if card != "J":
            alt_hand = hand.replace("J", card)
            alt_hand = sort_hand(alt_hand)
            alt_type = find_type(alt_hand)
            alt_strength = type_strength[alt_type]
            #print(hand, alt_hand, alt_type, alt_strength)
            opt_hand = max_of_two_hands(opt_hand, alt_hand, opt_strength, alt_strength, joker = True)
            if opt_hand == alt_hand:
                opt_strength = alt_strength
    return opt_hand, opt_strength

def add_optimize(d_hands):
    for hand in d_hands:
        d_hands[hand]["opt"], d_hands[hand]["strength"] = optimize_hand(hand, d_hands)
    return d_hands

            
# Main Program starts here
            
d_hands = get_data("Puzzle.txt")
d_hands = add_sort_hands(d_hands)
d_hands = add_type(d_hands)
d_hands = add_strength(d_hands)
d_hands = add_rank(d_hands, joker = False)
    
winning = 0
for hand in d_hands:
    winning += d_hands[hand]["bid"] * d_hands[hand]["rank"]
print("Puzzle 1: ", winning)


#for hand in d_hands:
#   print(hand, d_hands[hand])
    
d_hands = add_optimize(d_hands)
d_hands = add_rank(d_hands, joker = True)

#for hand in d_hands:
#    print(hand, d_hands[hand])
    
winning = 0
for hand in d_hands:
    winning += d_hands[hand]["bid"] * d_hands[hand]["rank"]
print("Puzzle 2: ", winning)

    

Puzzle 1:  252656917
Puzzle 2:  253499763
